In [1]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
    from selenium.webdriver.chrome.options import Options
except Exception as e:
    print(e)

In [2]:
data = db.DataBase()

In [3]:
def getData(ano, formacao, inicio=0, limitePaginas=None, curriculo=1):
    op = Options()
    op.headless = True
    navegador = webdriver.Chrome(options=op)
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")

    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscarEstrangeiros"]').click()
            
    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2
    troca = True
    
    if limitePaginas==None:
        limitePaginas = int(navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]').get_attribute("innerHTML").splitlines()[0])
        limitePaginas = int(paginasTotais/10 if paginasTotais%10==0 else (paginasTotais/10)+1)
    
    while(True):
        #Troca de pagina
        if inicio!=0:
            navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
            pagina = inicio
            link = navegador.current_url.split(";")
            link[0] = link[0].replace(str(10), str((pagina-1)*10))
            link = link[0]+";"+link[1]
            
            pag = inicio+1
            if inicio<11:
                pag = inicio+1
            elif inicio%10==0:
                pag = 13
            else:
                div = "1"+("0"*(len(str(inicio))-1))
                pag = int((inicio+1)/int(div))+2
            navegador.get(link)
            troca = False
            inicio = 0
        
        if troca:
            try:
                if pagina!=1 and pag<=11 and pagina<=11:
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<11 else 3

                if pagina>11 and pag>2:
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<13 else 3
            except Exception as e:
                print(e)
                break

        #Limitador de pagínas
        if limitePaginas != None and pagina == limitePaginas:
            break

        time.sleep(5)
        
        troca = True
        endFor = False
        for i in range(curriculo,11):
            try:
                #Selecionar curriculo
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                time.sleep(5)

                #Abrir curriculo
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()
            
            except:
                endFor = True
                break
                
            time.sleep(5)

            try:
                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

            except:
                navegador.switch_to.window(janelaAtual)
                navegador.get(navegador.current_url)
                i -= 1
                continue
            
            try:
                #Pegar dados 
                site = BeautifulSoup(navegador.page_source, 'html.parser')
                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if "2021" in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text
            except:
                navegador.close()
                time.sleep(1)
                #navegador.switch_to.window(janelaAtual)
                i-=1
                continue
            
            #Inserir no banco de dados
            if artigos != []:
                data.insertData(nome, formacao, artigos, ano)

            navegador.close()

            time.sleep(1)

            navegador.switch_to.window(janelaAtual)

            #Fechar curriculo
            navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
        
        if endFor:
            break
        
        data.connection.commit()
        pagina += 1
        curriculo = 1
        
    navegador.quit()
    if pagina==limitePaginas:
        return pagina, i
    else:
        return getData(ano, formacao, inicio=pagina, curriculo=i, limitePaginas=limitePaginas)

In [ ]:
try:
    p18, i18 = getData("2018", "doutorado", limitePaginas=100)
except Exception as e:
    print(e)

try:
    p19, i19 = getData("2019", "doutorado", limitePaginas=100)
except Exception as e:
    print(e)

In [14]:
data.selectData(2018)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Paulo,Paulo Broniera Junior,doutorado,EEG-FES-Force-MMG closed-loop control systems...,2018
1,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,Avaliação da compacidade em habitações com di...,2018
2,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,PHYSICAL AND MECHANICAL CHARACTERIZATION OF P...,2018
3,Adalberon,Adalberon Moreira de Lima Filho,doutorado,Análise dos projetos pedagógicos dos cursos d...,2018
4,Adalberon,Adalberon Moreira de Lima Filho,doutorado,¿De Olho no Velho Chico¿: uma proposta de jog...,2018
...,...,...,...,...,...
346,Paulo,Paulo Roberto Figueira Leal,doutorado,cultura participativa e o reordenamento da es...,2018
347,Paulo,Paulo Brito do Prado,doutorado,Em busca de outros passados museológicos: con...,2018
348,Vinícius,Vinícius Ynoe de Moraes,doutorado,Nonsurgical Treatment for Symptomatic Carpal ...,2018
349,Elisângela,Elisângela Claudia de Medeiros Moreira,doutorado,MAICON DE ARAUJO ; Cinema and interfaces in ...,2018


In [13]:
data.selectData(2019)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Galdino,Galdino Luiz Ramos Junior,doutorado,A inserção definitiva do direito civil e proc...,2019
1,Galdino,Galdino Luiz Ramos Junior,doutorado,QUEM SÃO OS DONOS DO PODER?,2019
2,Anderson,Anderson Mutter Teixeira,doutorado,The transmission mechanisms of macroprudentia...,2019
3,Anderson,Anderson Mutter Teixeira,doutorado,Prudential measures and their adverse effects...,2019
4,Maciel,Maciel Manoel de Queiroz,doutorado,Social capital and individual entrepreneurial...,2019
...,...,...,...,...,...
478,Cássio,Cássio do Nascimento,doutorado,Comparison of oral microbiome profile of poly...,2019
479,Cássio,Cássio do Nascimento,doutorado,Clinical study of the biofilm of implant-supp...,2019
480,Cássio,Cássio do Nascimento,doutorado,Assessment of virulence mechanisms of Pseudom...,2019
481,Daniella,Daniella de Souza e Silva,doutorado,INTERNATIONAL JOURNAL OF BIOLOGICAL MACROMOLE...,2019
